In [8]:
# type: ignore

import pandas as pd
import water.utils.parse as p
import os
import numpy as np
import boto3


In [ ]:
importlib.reload(p)

# Copying to folder in standard format

In [ ]:
for root, dirs, files in os.walk(p.folder_raw):
    for file in files:
        if file.endswith(".pdf"):
            # Copy the file to the folder_new
            filename_new = p.folder_new + '/' + file
            # Get the old file, expanding the sub-path
            filename_old = os.path.join(root, file)
            # Copy from old to new if new does not exist
            if not os.path.exists(filename_new):
                print(filename_old)
                os.system('cp ' + filename_old + ' ' + filename_new)   

/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_05_03.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_05_17.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_02_22.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_07_06.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_07_12.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_09_17.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_09_03.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_10_22.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_10_23.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_09_02.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_09_16.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_07_13.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_07_07.pdf
/Users/cg/Dropbox/code/Python/water/data/input/reserva/2024_02_23.pdf
/Users/cg/Dropbox/co

# Parsing to data

Note: This takes about 1 min per 10 files

In [23]:
import importlib
importlib.reload(p)

filenames_raw = sorted(os.listdir(folder_input))
# filenames = [f for f in filenames_raw if p.filter_weekly(f)]
filenames = filenames_raw[-300:][::-1]
filenames

['2024_11_08.pdf',
 '2024_11_07.pdf',
 '2024_11_06.pdf',
 '2024_11_05.pdf',
 '2024_11_04.pdf',
 '2024_11_03.pdf',
 '2024_11_02.pdf',
 '2024_11_01.pdf',
 '2024_10_31.pdf',
 '2024_10_30.pdf',
 '2024_10_29.pdf',
 '2024_10_28.pdf',
 '2024_10_27.pdf',
 '2024_10_26.pdf',
 '2024_10_25.pdf',
 '2024_10_24.pdf',
 '2024_10_23.pdf',
 '2024_10_22.pdf',
 '2024_10_21.pdf',
 '2024_10_20.pdf',
 '2024_10_19.pdf',
 '2024_10_18.pdf',
 '2024_10_17.pdf',
 '2024_10_16.pdf',
 '2024_10_15.pdf',
 '2024_10_14.pdf',
 '2024_10_13.pdf',
 '2024_10_12.pdf',
 '2024_10_11.pdf',
 '2024_10_10.pdf',
 '2024_10_09.pdf',
 '2024_10_08.pdf',
 '2024_10_07.pdf',
 '2024_10_06.pdf',
 '2024_10_05.pdf',
 '2024_10_04.pdf',
 '2024_10_03.pdf',
 '2024_10_02.pdf',
 '2024_10_01.pdf',
 '2024_09_30.pdf',
 '2024_09_29.pdf',
 '2024_09_28.pdf',
 '2024_09_27.pdf',
 '2024_09_26.pdf',
 '2024_09_25.pdf',
 '2024_09_24.pdf',
 '2024_09_23.pdf',
 '2024_09_22.pdf',
 '2024_09_21.pdf',
 '2024_09_20.pdf',
 '2024_09_19.pdf',
 '2024_09_18.pdf',
 '2024_09_17

In [24]:
for i, filename in enumerate(filenames):
    if i % 50 == 0:
        print(f'{i} of {len(filenames)}: {filename} / share completed: {round(i/len(filenames)*100, 2)}%')
    full_filename = os.path.join(folder_new, filename)
    filename_output = filename.replace('pdf', 'csv')
    filename_output = os.path.join(folder_output, filename_output)
    if not os.path.exists(filename_output):
        try:
            tables = p.get_tables(full_filename)
            df_raw = p.get_df(tables)
            df = p.clean_df(df_raw)
            df.to_csv(filename_output, index=False)
        except Exception as e:
            print(full_filename)
            print(e)
            pass

0 of 300: 2024_11_08.pdf / share completed: 0.0%
50 of 300: 2024_09_19.pdf / share completed: 16.67%
100 of 300: 2024_07_31.pdf / share completed: 33.33%
/Users/cg/Dropbox/code/Python/water/data/input/cleaned/2024_07_31.pdf
Cannot read an empty file
150 of 300: 2024_06_11.pdf / share completed: 50.0%
200 of 300: 2024_04_22.pdf / share completed: 66.67%
250 of 300: 2024_03_03.pdf / share completed: 83.33%


In [31]:
importlib.reload(p)

p.folder_output

'/Users/cg/Dropbox/code/Python/water/data/output'

# Creating cleaned bulk data

['2012_08_31.csv',
 '2012_09_01.csv',
 '2012_09_11.csv',
 '2012_09_21.csv',
 '2012_10_01.csv',
 '2012_10_11.csv',
 '2012_10_21.csv',
 '2012_10_31.csv',
 '2012_11_01.csv',
 '2012_11_11.csv',
 '2012_11_21.csv',
 '2012_12_01.csv',
 '2012_12_11.csv',
 '2012_12_21.csv',
 '2012_12_31.csv',
 '2013_01_01.csv',
 '2013_01_11.csv',
 '2013_01_21.csv',
 '2013_01_31.csv',
 '2013_02_01.csv',
 '2013_02_11.csv',
 '2013_02_21.csv',
 '2013_03_01.csv',
 '2013_03_11.csv',
 '2013_03_21.csv',
 '2013_03_31.csv',
 '2013_04_01.csv',
 '2013_04_11.csv',
 '2013_04_21.csv',
 '2013_05_01.csv',
 '2013_05_11.csv',
 '2013_05_21.csv',
 '2013_05_31.csv',
 '2013_06_01.csv',
 '2013_06_11.csv',
 '2013_06_21.csv',
 '2013_07_01.csv',
 '2013_07_11.csv',
 '2013_07_21.csv',
 '2013_07_31.csv',
 '2013_08_01.csv',
 '2013_08_11.csv',
 '2013_08_21.csv',
 '2013_08_31.csv',
 '2013_09_01.csv',
 '2013_09_11.csv',
 '2013_09_21.csv',
 '2013_10_01.csv',
 '2013_10_11.csv',
 '2013_10_21.csv',
 '2013_10_31.csv',
 '2013_11_01.csv',
 '2013_11_11

In [ ]:
filenames_all = sorted(os.listdir(p.folder_output))
df_all = p.get_full_df(filenames_all)
df_all.to_csv(f'{p.folder_base}/all_parsed.csv', index=False)
df_all = p.correct_issues(df_all).sort_values(['ds', 'province', 'reservoir'])

def remove_bad_rows(df):
    df_all = p.add_cols(df)

    df_all['date_lag'] = df_all.groupby(['province', 'reservoir'])['date'].shift(1)

    cols = ['rainfallsince', 'stored_hm3', 'capacity_hm3']
    for var in ['rainfallsince', 'stored_hm3']:
        df_all[f'{var}_diff'] = df_all.groupby(['province', 'reservoir'])[var].diff()
        df_all[f'{var}_diff_0'] = df_all[f'{var}_diff']
        for lags in range(1, 10):
            df_all[f'{var}_diff_{lags}'] = df_all.groupby(['province', 'reservoir'])[f'{var}_diff'].shift(lags)
            
    df_all['bad_data'] = df_all.rainfallsince_diff_0 < -10
    df_all['bad_data_for_year'] = df_all.groupby(['province', 'reservoir', 'year_climatic'])['bad_data'].transform('any')
    df_all['problem_month'] = df_all.month.isin([1, 10, 11, 12])
    df_all['bad_data_and_month'] = df_all.bad_data & df_all.problem_month

    df_all['stored_hm3_diff_relative'] = df_all.stored_hm3_diff / df_all.capacity_hm3
    df_reg = df_all.query('~bad_data_and_month').query('rainfallsince_diff >=0').copy()

    df_reg['bad_data_for_year'] = df_reg.groupby(['province', 'reservoir', 'year_climatic'])['bad_data'].transform('any')
    df_reg = df_reg.sort_values(['province', 'reservoir', 'date'])
    df_reg['date_diff'] = (df_reg.date - df_reg.date_lag).dt.days
    df_reg[['province', 'reservoir', 'date']].head(10)

    df_reg['bad_data_for_year'] = df_reg.groupby(['province', 'reservoir', 'year_climatic'])['bad_data'].transform('any')
    assert df_reg.bad_data_for_year.sum() == 0

    def add_lags(df, var_name, lags=np.arange(-5, 5), groups=['province', 'reservoir']):
        for lag in lags:
            df[f'{var_name}_lag_{lag}'] = df.groupby(groups)[var_name].shift(lag)
            
        lag_vars = [f'{var_name}_lag_{lag}' for lag in lags]
        return df, lag_vars

    df_reg['suspicious_storage'] = (np.abs(df_reg['stored_hm3_diff']) > 2) & (np.abs(df_reg['stored_hm3_diff_relative']) > 0.05)
    df_reg['high_rain'] = (df_reg.rainfallsince_diff > 10) | (df_reg.rainfallsince_diff_1 > 10)
    df_reg['bad_storage'] = df_reg.suspicious_storage & (~df_reg.high_rain)
    df_reg, lag_vars = add_lags(df_reg, 'bad_storage')

    df_reg['surrounding_bad_storage'] = df_reg[lag_vars].max(axis=1)
    df_reg = df_reg.query('surrounding_bad_storage==0').copy()
    return df_reg

df_removed = remove_bad_rows(df_all).sort_values(['ds', 'province', 'reservoir'])

df_removed = df_removed[df_all.columns].sort_values(['reservoir', 'ds'])

len(df_removed), len(df_all)
df_removed.to_csv(f'{folder_base}/all_parsed_cleaned.csv', index=False)
len(df_removed)

61714

In [33]:
importlib.reload(p)

<module 'water.utils.parse' from '/Users/cg/Dropbox/code/Python/water/app/backend/water/utils/parse.py'>

In [35]:
def pick_monthly(df):
    df_monthly = df[df.ds.str.slice(8, 10) == "01"].copy()
    return df_monthly

# Pick only the first day of the month from df_all
df_monthly = pick_monthly(df_all)
df_monthly_cleaned = pick_monthly(df_removed)

df_monthly.to_csv(f'{p.folder_base}/monthly.csv', index=False)
df_monthly_cleaned.to_csv(f'{p.folder_base}//monthly_cleaned.csv', index=False)


num_tail = 60
ds_recent = df_all.ds.unique()[-num_tail:]
df_recent = df_all[df_all.ds.isin(ds_recent)].copy()
df_recent_cleaned = df_removed[df_removed.ds.isin(ds_recent)].copy()

df_recent.to_csv(f'{p.folder_base}/recent.csv', index=False)
df_recent_cleaned.to_csv(f'{p.folder_base}/recent_cleaned.csv', index=False)

# Uploading bulk data to S3

In [ ]:
import shutil
BUCKET = 'andalucianwater'

s3 = boto3.client('s3')
filename_end = 'all_parsed_cleaned.csv'

filename = f'{p.folder_base}/{filename_end}'
folder_app = f'{p.cd_water}/app/backend/water/data_raw'

shutil.copy(filename, f'{folder_app}/{filename_end}')
folder_s3 = 'data/cleaned'
with open(filename, 'rb') as f:
    s3.upload_fileobj(f, BUCKET, f'{folder_s3}/{filename_end}')
    
file_geo = f'{p.folder_data}/reservoirs/InfGeografica/InfVectorial/Gpkg/Inv_presas_embalses.gpkg'
folder_geo_s3 = 'data/raw/reservoirs_geo'
with open(file_geo, 'rb') as f:
    s3.upload_fileobj(f, BUCKET, f'{folder_geo_s3}/reservoirs.gpkg')

ClientError: An error occurred (InvalidAccessKeyId) when calling the PutObject operation: The AWS Access Key Id you provided does not exist in our records.